In [3]:
from flipside import Flipside
import numpy as np
import pandas as pd 
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [4]:
""" This repo GITIGNORES api_key.txt, use this convention to store your key safely"""

with open('api_key.txt', 'r') as file:
    # Read the first line
    api_key = file.readline().strip()  # .strip() removes newline characters


"""Initialize Flipside with your API Key / API Url"""
flipside = Flipside(api_key, "https://api-v2.flipsidecrypto.xyz")


In [5]:
sql = """WITH RECURSIVE date_series AS (
  SELECT
    '2024-02-01' :: DATE AS day
  UNION
  ALL
  SELECT
    day + INTERVAL '1 day'
  FROM
    date_series
  WHERE
    day < CURRENT_DATE
),
traders AS (
  SELECT
    ORIGIN_FROM_ADDRESS,
    SUM(AMOUNT_OUT_USD) AS Volume
  FROM
    ethereum.defi.ez_dex_swaps
  GROUP BY
    ORIGIN_FROM_ADDRESS
  HAVING
    SUM(AMOUNT_OUT_USD) > 10000000
    AND MIN(BLOCK_TIMESTAMP) >= '2024-02-01'
    AND MIN(BLOCK_TIMESTAMP) < '2024-03-01'
  ORDER BY
    SUM(AMOUNT_OUT_USD) DESC
  LIMIT
    500
),
symbols AS (
  SELECT
    DISTINCT SYMBOL,
    COALESCE(
      CONTRACT_ADDRESS,
      '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
    ) AS token_contract
  FROM
    ethereum.core.ez_balance_deltas
  WHERE
    USER_ADDRESS IN (
      SELECT DISTINCT ORIGIN_FROM_ADDRESS FROM traders
    )
    AND HAS_PRICE = TRUE
),
daily_balances AS (
  SELECT
    DATE_TRUNC('day', BLOCK_TIMESTAMP) AS day,
    SYMBOL,
    CURRENT_BAL AS daily_balance,
    COALESCE(
      CONTRACT_ADDRESS,
      '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
    ) AS token_contract,
    BLOCK_TIMESTAMP,
    USER_ADDRESS
  FROM
    ethereum.core.ez_balance_deltas
  WHERE
    USER_ADDRESS IN (
      SELECT DISTINCT ORIGIN_FROM_ADDRESS FROM traders
    )
    AND HAS_PRICE = TRUE
),
ranked_balances AS (
  SELECT
    day,
    SYMBOL,
    daily_balance,
    token_contract,
    USER_ADDRESS,
    ROW_NUMBER() OVER (
      PARTITION BY token_contract, day
      ORDER BY BLOCK_TIMESTAMP DESC
    ) AS rn
  FROM
    daily_balances
),
filtered_balances AS (
  SELECT
    day,
    SYMBOL,
    daily_balance,
    token_contract,
    USER_ADDRESS
  FROM
    ranked_balances
  WHERE
    rn = 1
),
filled_balances AS (
  SELECT
    ds.day,
    s.SYMBOL,
    COALESCE(fb.daily_balance, 0) AS filled_balance,
    fb.USER_ADDRESS,
    CASE
      WHEN s.SYMBOL = 'ETH' THEN LOWER('0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2')
      ELSE fb.token_contract
    END AS token_contract
  FROM
    date_series ds
    CROSS JOIN symbols s
    LEFT JOIN filtered_balances fb ON fb.day <= ds.day
    AND fb.SYMBOL = s.SYMBOL QUALIFY ROW_NUMBER() OVER (
      PARTITION BY ds.day, s.SYMBOL
      ORDER BY fb.day DESC
    ) = 1
),
latest_prices AS (
  SELECT
    DATE_TRUNC('day', hour) AS day,
    TOKEN_ADDRESS,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY PRICE) AS price
  FROM
    ethereum.price.ez_prices_hourly
  GROUP BY
    DATE_TRUNC('day', hour), TOKEN_ADDRESS
),
balances_with_prices AS (
  SELECT
    fb.day,
    fb.SYMBOL,
    fb.filled_balance,
    USER_ADDRESS,
    lp.PRICE AS daily_price,
    fb.filled_balance * lp.PRICE AS value_in_usd
  FROM
    filled_balances fb
    LEFT JOIN latest_prices lp ON lp.TOKEN_ADDRESS = fb.token_contract
    AND lp.day = fb.day
),
aggregated_by_day AS (
  SELECT
    day,
    daily_price,
    SYMBOL,
    USER_ADDRESS,
    SUM(filled_balance) AS total_filled_balance,
    SUM(value_in_usd) AS total_value_in_usd
  FROM
    balances_with_prices
  GROUP BY
    day, SYMBOL, USER_ADDRESS, daily_price
),
aggregated_added AS (
  SELECT
    a.day,
    a.SYMBOL,
    a.daily_price,
    a.total_filled_balance,
    a.USER_ADDRESS,
    a.total_value_in_usd,
    SUM(a.total_value_in_usd) OVER (PARTITION BY a.day, a.USER_ADDRESS) AS total_usd_in_treasury
  FROM
    aggregated_by_day a
  WHERE
    a.day <= CURRENT_DATE - 1
    AND a.daily_price > 0
  ORDER BY
    a.day DESC, a.SYMBOL
),
aggregated_added2 AS (
  SELECT
    *,
    total_value_in_usd / NULLIF(total_usd_in_treasury, 0) AS composition
  FROM
    aggregated_added
  ORDER BY
    day DESC, SYMBOL, USER_ADDRESS
),
daily_returns AS (
  SELECT
    day,
    SYMBOL,
    daily_price,
    composition,
    total_filled_balance,
    total_usd_in_treasury,
    USER_ADDRESS,
    LAG(daily_price) OVER (PARTITION BY SYMBOL ORDER BY day) AS prev_day_price,
    (daily_price - LAG(daily_price) OVER (PARTITION BY SYMBOL ORDER BY day)) / LAG(daily_price) OVER (PARTITION BY SYMBOL ORDER BY day) AS daily_return
  FROM
    aggregated_added2
),
weighted_daily_returns AS (
  SELECT
    day,
    USER_ADDRESS,
    daily_return,
    SUM(daily_return * composition) AS weighted_daily_return
  FROM
    daily_returns
  GROUP BY
    day, USER_ADDRESS, daily_return
),
cumulative_returns AS (
  SELECT
    day,
    daily_return,
    weighted_daily_return,
    USER_ADDRESS,
    (1 + weighted_daily_return) AS daily_factor,
    EXP(SUM(LN(1 + weighted_daily_return)) OVER (PARTITION BY USER_ADDRESS ORDER BY day ASC)) - 1 AS cumulative_return
  FROM
    weighted_daily_returns
),
base_return AS (
  SELECT
    day,
    USER_ADDRESS,
    daily_return,
    cumulative_return AS base_cumulative_return
  FROM
    cumulative_returns
  WHERE
    cumulative_return IS NOT NULL
),
normalized_returns AS (
  SELECT
    day,
    USER_ADDRESS,
    daily_return,
    base_cumulative_return,
    100 + (100 * (base_cumulative_return - FIRST_VALUE(base_cumulative_return) OVER (PARTITION BY USER_ADDRESS ORDER BY day ASC))) AS portfolio_return
  FROM
    base_return
),
return_stats AS (
  SELECT
    day,
    USER_ADDRESS,
    base_cumulative_return,
    portfolio_return,
    AVG(portfolio_return) OVER (PARTITION BY USER_ADDRESS ORDER BY day ASC) AS avg_daily_return,
    STDDEV(portfolio_return) OVER (PARTITION BY USER_ADDRESS ORDER BY day ASC) AS stddev_return
  FROM
    normalized_returns
),
sharpe_ratio AS (
  SELECT
    day,
    USER_ADDRESS,
    base_cumulative_return,
    portfolio_return,
    NULLIF((portfolio_return - 0.0430 / 365) / NULLIF(STDDEV(portfolio_return) OVER (PARTITION BY USER_ADDRESS ORDER BY day ASC), 0), 0) AS daily_sharpe_ratio
  FROM
    return_stats
),
trade_counts AS (
  SELECT
    DATE_TRUNC('day', BLOCK_TIMESTAMP) AS day,
    ORIGIN_FROM_ADDRESS AS USER_ADDRESS,
    COUNT(DISTINCT tx_hash) AS number_of_trades
  FROM
    ethereum.defi.ez_dex_swaps
  GROUP BY
    day, ORIGIN_FROM_ADDRESS
),

tokens_traded AS (
  SELECT
    DATE_TRUNC('day', BLOCK_TIMESTAMP) AS day,
    ORIGIN_FROM_ADDRESS AS USER_ADDRESS,
    COUNT(DISTINCT SYMBOL_OUT) AS unique_tokens_traded
  FROM
    ethereum.defi.ez_dex_swaps
  GROUP BY
    day, ORIGIN_FROM_ADDRESS
),

final_normalized_returns AS (
  SELECT
    sr.day,
    sr.USER_ADDRESS,
    sr.base_cumulative_return,
    sr.portfolio_return,
    sr.daily_sharpe_ratio,
    CASE
      WHEN AVG(sr.portfolio_return) > 120 AND sr.daily_sharpe_ratio > 1.5 THEN 'Good Trader'
      WHEN AVG(sr.portfolio_return) BETWEEN 100 AND 120 THEN 'Average Trader'
      WHEN AVG(sr.portfolio_return) < 100 THEN 'Bad Trader'
    END AS trader_class
  FROM
    sharpe_ratio sr
  WHERE
    sr.daily_sharpe_ratio IS NOT NULL
  GROUP BY
    sr.day, sr.USER_ADDRESS, sr.base_cumulative_return, sr.portfolio_return, sr.daily_sharpe_ratio
)

SELECT
  fnr.day,
  fnr.USER_ADDRESS,
  fnr.base_cumulative_return,
  fnr.portfolio_return,
  fnr.daily_sharpe_ratio,
  fnr.trader_class,
  tc.number_of_trades,
  tt.unique_tokens_traded
FROM
  final_normalized_returns fnr
  LEFT JOIN trade_counts tc ON tc.day = fnr.day AND tc.USER_ADDRESS = fnr.USER_ADDRESS
  LEFT JOIN tokens_traded tt ON tt.day = fnr.day AND tt.USER_ADDRESS = fnr.USER_ADDRESS;
"""
"""Run the query against Flipside's query engine and await the results"""
query_result_set = flipside.query(sql)
#print(query_result_set)

In [6]:
"""This function will be added to Flipside package after testing, just copy/paste as needed for now"""
def auto_paginate_result(query_result_set, page_size=10000):
    """
    This function auto-paginates a query result to get all the data. It assumes 10,000 rows per page.
    In case of an error, reduce the page size. Uses numpy.
    """
    num_rows = query_result_set.page.totalRows
    page_count = np.ceil(num_rows / page_size).astype(int)
    all_rows = []
    current_page = 1
    while current_page <= page_count:
        results = flipside.get_query_results(
            query_result_set.query_id,
            page_number=current_page,
            page_size=page_size
        )
        if results.records:
            all_rows.extend(results.records)  # Use extend() to add list elements
        current_page += 1  # Increment the current page number
    return all_rows  # Return all_rows in JSON format


In [7]:
""" Get your data as a pandas data frame"""

trader_classifier = auto_paginate_result(query_result_set)
df = pd.DataFrame(trader_classifier)

# Set the display option to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)  # Adjusts the display width for better visibility

#Display Dataframe
print(df.head())

                        day                                user_address  base_cumulative_return  portfolio_return  daily_sharpe_ratio trader_class  number_of_trades  unique_tokens_traded  __row_index
0  2024-08-25T00:00:00.000Z  0xadb4baa4bf3a1c176c4399402395403632b036a3            38074.278177      3.807529e+06            2.052683  Good Trader               NaN                   NaN            0
1  2024-05-10T00:00:00.000Z  0x6e2c509d522d47f509e1a6d75682e6abbc38b362                0.780245      1.756767e+02            9.102561  Good Trader               1.0                   2.0            1
2  2024-02-22T00:00:00.000Z  0x3f8cff57fb4592a0ba46c66d2239486b8690842e               -0.095655      8.587805e+01           12.691943   Bad Trader               3.0                   3.0            2
3  2024-06-13T00:00:00.000Z  0x518ded2f413f275d720b199ad9d03a262e35d3b3               -0.520177      4.907032e+01            2.252786   Bad Trader               NaN                   NaN            3


In [44]:
#Drop the "__row_index" column
df_t = df.drop(columns = ['__row_index'])

#Remove later
print(df_t.head())

                        day                                user_address  base_cumulative_return  portfolio_return  daily_sharpe_ratio trader_class  number_of_trades  unique_tokens_traded
0  2024-08-25T00:00:00.000Z  0xadb4baa4bf3a1c176c4399402395403632b036a3            38074.278177      3.807529e+06            2.052683  Good Trader               NaN                   NaN
1  2024-05-10T00:00:00.000Z  0x6e2c509d522d47f509e1a6d75682e6abbc38b362                0.780245      1.756767e+02            9.102561  Good Trader               1.0                   2.0
2  2024-02-22T00:00:00.000Z  0x3f8cff57fb4592a0ba46c66d2239486b8690842e               -0.095655      8.587805e+01           12.691943   Bad Trader               3.0                   3.0
3  2024-06-13T00:00:00.000Z  0x518ded2f413f275d720b199ad9d03a262e35d3b3               -0.520177      4.907032e+01            2.252786   Bad Trader               NaN                   NaN
4  2024-03-18T00:00:00.000Z  0x725ef823a0c7ea654561e13796a9d81a9a

In [87]:
# Convert 'day' column to datetime format and remove time component 
df_t['day'] = pd.to_datetime(df_t['day'], errors='coerce').dt.date
#Remove later
print(df_t.head())

          day                                user_address  base_cumulative_return  portfolio_return  daily_sharpe_ratio trader_class  number_of_trades  unique_tokens_traded
0  2024-08-25  0xadb4baa4bf3a1c176c4399402395403632b036a3            38074.278177      3.807529e+06            2.052683  Good Trader               NaN                   NaN
1  2024-05-10  0x6e2c509d522d47f509e1a6d75682e6abbc38b362                0.780245      1.756767e+02            9.102561  Good Trader               1.0                   2.0
2  2024-02-22  0x3f8cff57fb4592a0ba46c66d2239486b8690842e               -0.095655      8.587805e+01           12.691943   Bad Trader               3.0                   3.0
3  2024-06-13  0x518ded2f413f275d720b199ad9d03a262e35d3b3               -0.520177      4.907032e+01            2.252786   Bad Trader               NaN                   NaN
4  2024-03-18  0x725ef823a0c7ea654561e13796a9d81a9aa8398a                0.005050      8.613312e+01            2.370263   Bad Trader   

In [88]:
# Define a function to detect outliers based on IQR
def find_outliers_iqr(df_t, column):
    Q1 = df_t[column].quantile(0.25)
    Q3 = df_t[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df_t[(df_t[column] < lower_bound) | (df_t[column] > upper_bound)]

#Find outliers for 'portfolio_return'
outliers = find_outliers_iqr(df_t, 'portfolio_return')
print(outliers)


              day                                user_address  base_cumulative_return  portfolio_return  daily_sharpe_ratio trader_class  number_of_trades  unique_tokens_traded
0      2024-08-25  0xadb4baa4bf3a1c176c4399402395403632b036a3            3.807428e+04      3.807529e+06            2.052683  Good Trader               NaN                   NaN
9      2024-07-01  0xfb94d3404c1d3d9d6f08f79e58041d5ea95accfa            2.247147e+00      3.250688e+02            4.866034  Good Trader               NaN                   NaN
15     2024-05-15  0x9c03bf3a8df813938becbdc42c8156f8964cdf46            4.937530e+01      5.037485e+03            2.806068  Good Trader               NaN                   NaN
16     2024-08-14  0x6f5884a6ba14297eab47aa52c4e739f0a11ff7c1            1.503903e+07      1.503903e+09            1.486914         None               NaN                   NaN
18     2024-06-20  0x1518c211f611114decd1842894e08b803afcd27b            7.978986e+00      8.749185e+02            

In [89]:
# Remove outliers from 'portfolio_return' column based on IQR method
def remove_outliers_iqr(df_t, column):
    Q1 = df_t[column].quantile(0.25)
    Q3 = df_t[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df_t[(df_t[column] >= lower_bound) & (df_t[column] <= upper_bound)]

# Apply the function to remove outliers from the 'portfolio_return' column
df_clean = remove_outliers_iqr(df_t, 'portfolio_return')

# Verify the result
print(df_clean)


              day                                user_address  base_cumulative_return  portfolio_return  daily_sharpe_ratio    trader_class  number_of_trades  unique_tokens_traded
1      2024-05-10  0x6e2c509d522d47f509e1a6d75682e6abbc38b362                0.780245        175.676668            9.102561     Good Trader               1.0                   2.0
2      2024-02-22  0x3f8cff57fb4592a0ba46c66d2239486b8690842e               -0.095655         85.878050           12.691943      Bad Trader               3.0                   3.0
3      2024-06-13  0x518ded2f413f275d720b199ad9d03a262e35d3b3               -0.520177         49.070325            2.252786      Bad Trader               NaN                   NaN
4      2024-03-18  0x725ef823a0c7ea654561e13796a9d81a9aa8398a                0.005050         86.133115            2.370263      Bad Trader               3.0                   3.0
5      2024-06-16  0x725ef823a0c7ea654561e13796a9d81a9aa8398a               -0.524878         33.140

In [90]:
# Check for NaN values in the entire DataFrame
print(df_clean.isna().sum())

day                           0
user_address                  0
base_cumulative_return        0
portfolio_return              0
daily_sharpe_ratio            0
trader_class                  8
number_of_trades          10660
unique_tokens_traded      10660
dtype: int64


In [91]:
#drop rows in trader_class column with NaN values
df_cleann = df_clean.dropna(subset=['trader_class'])
print(df_cleann.isna().sum())

day                           0
user_address                  0
base_cumulative_return        0
portfolio_return              0
daily_sharpe_ratio            0
trader_class                  0
number_of_trades          10653
unique_tokens_traded      10653
dtype: int64


In [92]:
#Fill NaN with 0
df_cleaned = df_cleann.fillna(0)
print(df_cleaned.isna().sum())

day                       0
user_address              0
base_cumulative_return    0
portfolio_return          0
daily_sharpe_ratio        0
trader_class              0
number_of_trades          0
unique_tokens_traded      0
dtype: int64


In [93]:
df_cleaned.dtypes

day                        object
user_address               object
base_cumulative_return    float64
portfolio_return          float64
daily_sharpe_ratio        float64
trader_class               object
number_of_trades          float64
unique_tokens_traded      float64
dtype: object

In [95]:
#convert day to datetime
df_cleaned['day'] = pd.to_datetime(df_cleaned['day'])

In [96]:
df_cleaned.dtypes

day                       datetime64[ns]
user_address                      object
base_cumulative_return           float64
portfolio_return                 float64
daily_sharpe_ratio               float64
trader_class                      object
number_of_trades                 float64
unique_tokens_traded             float64
dtype: object

In [97]:
df_cleaned.trader_class.head()

1    Good Trader
2     Bad Trader
3     Bad Trader
4     Bad Trader
5     Bad Trader
Name: trader_class, dtype: object

In [101]:
# Convert 'trader_class' to a categorical column
df_cleaned["trader_class"] = df_cleaned["trader_class"].astype('category')
df_cleaned["trader_class"].cat.categories

Index(['Average Trader', 'Bad Trader', 'Good Trader'], dtype='object')

In [102]:
df_cleaned.dtypes

day                       datetime64[ns]
user_address                      object
base_cumulative_return           float64
portfolio_return                 float64
daily_sharpe_ratio               float64
trader_class                    category
number_of_trades                 float64
unique_tokens_traded             float64
dtype: object

In [107]:
# Assign unique integers to categories
df_cleaned['trader_class_numeric'] = df_cleaned['trader_class'].astype('category').cat.codes
df_cleaned['trader_class_numeric']

1        2
2        1
3        1
4        1
5        1
        ..
17813    0
17814    1
17815    2
17817    0
17819    1
Name: trader_class_numeric, Length: 14463, dtype: int8

In [106]:
df_cleaned.head()

,day,user_address,base_cumulative_return,portfolio_return,daily_sharpe_ratio,trader_class,number_of_trades,unique_tokens_traded,trader_class_numeric
1,2024-05-10,0x6e2c509d522d47f509e1a6d75682e6abbc38b362,0.780245,175.676668,9.102561,Good Trader,1.0,2.0,2
2,2024-02-22,0x3f8cff57fb4592a0ba46c66d2239486b8690842e,-0.095655,85.878050,12.691943,Bad Trader,3.0,3.0,1
3,2024-06-13,0x518ded2f413f275d720b199ad9d03a262e35d3b3,-0.520177,49.070325,2.252786,Bad Trader,0.0,0.0,1
4,2024-03-18,0x725ef823a0c7ea654561e13796a9d81a9aa8398a,0.005050,86.133115,2.370263,Bad Trader,3.0,3.0,1
5,2024-06-16,0x725ef823a0c7ea654561e13796a9d81a9aa8398a,-0.524878,33.140320,0.786522,Bad Trader,0.0,0.0,1
